# **[Diff-SVC](https://github.com/prophesier/diff-svc)**
Singing Voice Conversion via diffusion model

____

####  **Notebook put together by [justinjohn-03](https://github.com/justinjohn0306)**
#### **Edited for batch rendering by [MLo7](https://github.com/MLo7Ghinsan)**

## **Special thanks to [prophesier](https://github.com/prophesier) and [UtaUtaUtau](https://github.com/UtaUtaUtau)**

In [ ]:
#@markdown ## Mount your Gdrive
#@markdown (This is an essential step if you want to load your own trained model)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title # Setup
#@markdown ## Install Diff-SVC
from IPython.display import clear_output
from google.colab import files
import os

!rm -rf /content/sample_data


Mode = "install" #@param ["install", "update", "remove"]
Repository = "UtaUtaUtau" #@param ["Official Diff-SVC", "UtaUtaUtau"]
Branch_name = "" #@param {type:"string"}

repositories = {
  'Official Diff-SVC':'prophesier',
  'UtaUtaUtau':'UtaUtaUtau'
}

from pathlib import Path
if Mode == 'install':
  git_cmd = ''
  if Branch_name: git_cmd += f"-b {Branch_name} "

  git_cmd += f"--depth 1 https://github.com/{repositories[Repository]}/diff-svc.git"
  !git clone $git_cmd
  %cd /content/diff-svc
  print('Installing torch')
  !pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
  !pip install -r requirements_short.txt
  !pip install tensorboard<2.9,>=2.8
  %reload_ext tensorboard
  print('Downloading pretrained models')
  %cd "/content/"
  %mkdir -p /content/diff-svc/checkpoints/
  !wget https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/diff-svc-necessary-checkpoints/0102_xiaoma_pe.zip
  !wget https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/diff-svc-necessary-checkpoints/0109_hifigan_bigpopcs_hop128.zip
  !wget https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/diff-svc-necessary-checkpoints/nsf_hifigan.zip
  !wget https://github.com/MLo7Ghinsan/MLo7_Diff-SVC_models/releases/download/diff-svc-necessary-checkpoints/hubert.zip
  !mkdir /content/diff-svc/checkpoints/0102_xiaoma_pe
  !mkdir /content/diff-svc/checkpoints/0109_hifigan_bigpopcs_hop128
  !mkdir /content/diff-svc/checkpoints/nsf_hifigan
  !mkdir /content/diff-svc/checkpoints/hubert
  !unzip /content/0102_xiaoma_pe.zip -d /content/diff-svc/checkpoints/0102_xiaoma_pe
  !unzip /content/0109_hifigan_bigpopcs_hop128.zip -d /content/diff-svc/checkpoints/0109_hifigan_bigpopcs_hop128
  !unzip /content/nsf_hifigan.zip -d /content/diff-svc/checkpoints/nsf_hifigan
  !unzip /content/hubert.zip -d /content/diff-svc/checkpoints/hubert

  !rm /content/0102_xiaoma_pe.zip
  !rm /content/0109_hifigan_bigpopcs_hop128.zip
  !rm /content/nsf_hifigan.zip
  !rm /content/hubert.zip

  !rm /content/diff-svc/results/test_output.wav


  clear_output()

  print('Done!')

elif Mode == 'update':
  %cd /content/diff-svc
  !git pull
  !pip install -r requirements_short.txt
  clear_output()
  print("Done!")
else:
  answer = input("Are you sure you want to delete diff-svc folder? (y/n)").lower()
  while answer not in ["y", "n"]:
    print("Invalid input")
    answer = input("Are you sure you want to delete diff-svc folder? (y/n)").lower()
  if answer == "y":
    %cd /content
    %rm -r diff-svc/
    print("Done!")
  else:
    print("Cancelled...")

Done!


In [ ]:
!git clone https://github.com/zshicode/GNN-for-text-classification.git
%cd GNN-for-text-classification/
%ls

Cloning into 'GNN-for-text-classification'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 36 (delta 3), reused 31 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), 1.52 MiB | 4.75 MiB/s, done.
/content/GNN-for-text-classification
config.py  data/  main.py  models/  preprocess/  README.md  result/  utils/


In [ ]:
# ! pip install pandas
# ! pip install hparams
# !pip list
!pip install numpy==1.23.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycwt 0.4.0b0 requires numpy<2,>=1.24, but you have numpy 1.23.3 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.12.1+cu113 which is incompatible.


In [ ]:
import os

In [ ]:
#@title # **Load model**

#@markdown ### **Load the pretrained model (default)**


#@markdown ___

#@markdown Note: Add the full path to the most recent checkpoint located on your Gdrive as well as the speaker's name if you wish to use your own model.

#@markdown Example:-

#@markdown The ``project_name`` will be the name of your speaker

#@markdown  ``model_path: /content/drive/MyDrive/Diff-SVC/checkpoints/model_name/model_ckpt_steps_50000.ckpt``

#@markdown           ``config_path: /content/drive/MyDrive/Diff-SVC/checkpoints/model_name/config.yaml``


#@markdown ___

#@markdown ### **Set model location with the name of the speaker:**
#@markdown *If you wish to use the pre-trained model and don't have your own model, leave these at their default values.*

#@markdown ___

%cd "/content/diff-svc/"

os.environ['PYTHONPATH']='.'

!CUDA_VISIBLE_DEVICES=0


from utils.hparams import hparams
from preprocessing.data_gen_utils import get_pitch_parselmouth,get_pitch_crepe
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import utils
import librosa
import torchcrepe
from infer import *
import logging
from infer_tools.infer_tool import *

logging.getLogger('numba').setLevel(logging.WARNING)

# 工程文件夹名，训练时用的那个
project_name = "MS" #@param {type: "string"}
model_path = "/content/drive/MyDrive/kubig/audio_project/MS_voice/model_ckpt_steps_112000.ckpt" #@param {type: "string"}
config_path="/content/drive/MyDrive/kubig/audio_project/MS_voice/config.yaml" #@param {type: "string"}
hubert_gpu=True
svc_model = Svc(project_name,config_path,hubert_gpu, model_path)
print('model loaded')

/content/diff-svc
| load 'model' from '/content/drive/MyDrive/kubig/audio_project/MS_voice/model_ckpt_steps_112000.ckpt'.
| load 'model' from 'checkpoints/0102_xiaoma_pe/model_ckpt_steps_60000.ckpt'.
| Load HifiGAN:  checkpoints/nsf_hifigan/model
Removing weight norm...
model loaded


In [ ]:
#@markdown ## Upload your reference audio

%cd "/content/diff-svc/raw/"

print("\n\033[34m\033[1mupload your reference audio")
listfn, length = files.upload().popitem()

%cd "/content/diff-svc/"
print("\n\033[32m\033[1mdone")

/content/diff-svc/raw

upload your reference audio


NameError: ignored

In [ ]:
# batch_audio 내부 파일 삭제
def delete_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} 파일이 삭제되었습니다.")
    else:
        print(f"{file_path} 파일이 존재하지 않습니다.")


In [ ]:
from glob import glob

In [ ]:
# remove batch_audio files
for target_file in glob('/content/diff-svc/batch_audio/*'):
#  print(target_file)
  delete_file(target_file)

/content/diff-svc/batch_audio/fearless_11.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_3.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_6.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_2.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_14.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_4.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_8.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_9.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_1.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_wav.zip 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_5.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_10.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/desktop.ini 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_7.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_12.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_13.wav 파일이 삭제되었습니다.
/content/diff-svc/batch_audio/fearless_0.wav 파일이 삭제되었습니다.


In [ ]:
#@markdown # Import your audio

%cd "/content/diff-svc"

#@markdown File location in Drive

#@markdown Or you can simply drag and drop your files into batch_audio folder after running this cell (remove the directory in this cell first)
audio_location = '/content/diff-svc/batch_audio/fearless2.zip' #@param {type: "string"}
!mkdir -p batch_audio
audio_clone = "batch_audio"
!rm /raw/test_input.wav
if audio_location.endswith('.rar'):
    !unrar x "$audio_location" "$audio_clone"
elif audio_location.endswith('.zip'):
    !unzip "$audio_location" -d "$audio_clone"
elif audio_location.endswith('.tar'):
    !tar -xf "$audio_location" -C "$audio_clone"
elif audio_location.endswith('.tar.gz'):
    !tar -xzf "$audio_location" -C "$audio_clone"
elif audio_location.endswith('.tar.bz2'):
    !tar -xjf "$audio_location" -C "$audio_clone"
else:
    !7za x "$audio_location" -o$audio_clone

# clear_output()

/content/diff-svc
rm: cannot remove '/raw/test_input.wav': No such file or directory
Archive:  /content/diff-svc/batch_audio/fearless2.zip
  inflating: batch_audio/0.wav       
  inflating: batch_audio/1.wav       
  inflating: batch_audio/10.wav      
  inflating: batch_audio/11.wav      
  inflating: batch_audio/12.wav      
  inflating: batch_audio/13.wav      
  inflating: batch_audio/14.wav      
  inflating: batch_audio/2.wav       
  inflating: batch_audio/3.wav       
  inflating: batch_audio/4.wav       
  inflating: batch_audio/5.wav       
  inflating: batch_audio/6.wav       
  inflating: batch_audio/7.wav       
  inflating: batch_audio/8.wav       
  inflating: batch_audio/9.wav       


위 블록 실행시에 batch_audio에 파일들 있어야함.

In [ ]:
#@title # Input audio and adjust parameters

#@markdown ### <b><font color="red"> Make sure that your imported audio have no subfolders before running this cell

#@markdown ___

#@markdown #### ``This shifts the raw audio up by one semitone before rendering, if the raw input is of a male voice and the desired voice is female, you can input 8 or 12 etc (12 would shift a whole octave).``
key = 8#@param {type: "integer"}
# 加速倍数
#@markdown ___

#@markdown #### ``The multiple of the inference acceleration , the default value is 1000 steps, inputting a value if 10 would mean only using 100 steps to render, it's a rather straightforward value. The value can go up to 50x (rendering in 20 steps) without causing audible quality loss, if the value is set any higher it may start to cause quality loss.``

#@markdown ___


#@markdown #### Note: ``If use_gt_mel is set to True below, you should keep this value lower than the add_noise_step value and keep it at a value where it can completely divide 1000.``


pndm_speedup = 10 #@param {type: "integer"}

#@markdown ___

#@markdown #### ``Related to the use_gt_mel parameter, it controls the balance of the input and target voice, a value of 1 is completely the raw input, a value of 1000 is completely the target voice, there's an audible mix in tone when the value falls around 300 (this value isn't linear, also, if this parameter is set very low, you can decrease the pndm exceleration value for higher rendering quality)``
add_noise_step = 1000 #@param {type: "integer"}
#@markdown ___
#@markdown #### ``Crepe's noise filter threshold, you can increase the value of the raw audio is clean, and if there is a lot of noise, you can keep or decrease the value, changing the use_crepe parameter to False will disable this parameter.``
thre = 1 #@param {type: "integer"}
#@markdown ___
#@markdown #### ``Crepe is a F0 calculation algorithm, it's good but slow, setting the value to False will change the F0 calculation algorithm from crepe to parselmouth that is faster than crepe but is of lower quality``
use_crepe= False #@param {type:"boolean"}
#@markdown ___
#@markdown #### ``F0 extraction algorithm for MEL spectogram rendering, using False will use the raw input's F0 for rendering. There's usually a difference in output between using True and False for rendering, usually setting it to True yields better results, but it's not set in stone, either value doesn't impact rendering speeds much. (Whatever the key value is, this is always changeable, doesn't affect it)``
use_pe=True #@param {type:"boolean"}
#@markdown ___
#@markdown #### ``This option is similar to the image-to-image function of AI art generation, if set to True, the output audio shall be a mix of the input voice and the target voice, the percentage of each is decided by the next parameter.``

#@markdown #### ``NOTE!!!: If this parameter is set to true, keep the key parameter value at 0, as rendering with various pitch input is not supported.``
use_gt_mel= False #@param {type:"boolean"}
#@markdown ___
#@markdown #### ``The folder of your audio files, default is batch_audio``
folder = "/content/diff-svc/batch_audio" #@param {type: "string"}
#@markdown ___
for name in os.listdir(folder):
    if name.endswith(".wav"):
        wav_fn = os.path.join(folder, name)
        print(wav_fn)
        out_folder = "results"
        wav_gen = os.path.join(out_folder, name)
        f0_tst, f0_pred, audio = run_clip(svc_model,file_path=wav_fn, key=key, acc=pndm_speedup, use_crepe=use_crepe, use_pe=use_pe, thre=thre,
                                        use_gt_mel=use_gt_mel, add_noise_step=add_noise_step,project_name=project_name,out_path=wav_gen)

/content/diff-svc/batch_audio/7.wav
code version:2022-12-04
executing 'slice' costed 1.027s
#=====segment start, 24.0s======
executing 'get_pitch' costed 8.670s
hubert (on cuda) time used 0.3849294185638428


sample time step: 100%|██████████| 100/100 [00:01<00:00, 63.10it/s]


executing 'diff_infer' costed 1.592s
executing 'after_infer' costed 0.756s
/content/diff-svc/batch_audio/2.wav
code version:2022-12-04
executing 'slice' costed 1.028s
#=====segment start, 24.0s======
executing 'get_pitch' costed 7.917s
hubert (on cuda) time used 0.4060184955596924


sample time step: 100%|██████████| 100/100 [00:01<00:00, 62.83it/s]


executing 'diff_infer' costed 1.600s
executing 'after_infer' costed 0.755s
/content/diff-svc/batch_audio/5.wav
code version:2022-12-04
executing 'slice' costed 1.050s
#=====segment start, 18.711s======
executing 'get_pitch' costed 7.368s
hubert (on cuda) time used 0.3909585475921631


...

executing 'diff_infer' costed 1.570s
executing 'after_infer' costed 0.757s
/content/diff-svc/batch_audio/3.wav
code version:2022-12-04
executing 'slice' costed 1.084s
#=====segment start, 24.0s======
executing 'get_pitch' costed 8.527s
hubert (on cuda) time used 0.36325550079345703


sample time step: 100%|██████████| 100/100 [00:01<00:00, 63.99it/s]


executing 'diff_infer' costed 1.570s
executing 'after_infer' costed 0.756s
/content/diff-svc/batch_audio/8.wav
code version:2022-12-04
executing 'slice' costed 1.073s
#=====segment start, 24.0s======
executing 'get_pitch' costed 8.528s
hubert (on cuda) time used 0.37801289558410645


sample time step: 100%|██████████| 100/100 [00:01<00:00, 64.06it/s]


executing 'diff_infer' costed 1.570s
executing 'after_infer' costed 0.756s


In [ ]:
#@markdown # Zip up the result to your drive

#@markdown ___

zip_exp_name = "fearless_MS_112000_8keyup_32bit" #@param {type:"string"}

!mkdir /content/drive/MyDrive/Diff-SVC-RESULTS
!zip -r "/content/drive/MyDrive/Diff-SVC-RESULTS/{zip_exp_name}.zip" "results"

mkdir: cannot create directory ‘/content/drive/MyDrive/Diff-SVC-RESULTS’: File exists
  adding: results/ (stored 0%)
  adding: results/fearless_11.wav (deflated 11%)
  adding: results/fearless_3.wav (deflated 11%)
  adding: results/fearless_6.wav (deflated 12%)
  adding: results/fearless_2.wav (deflated 9%)
  adding: results/fearless_14.wav (deflated 64%)
  adding: results/fearless_4.wav (deflated 11%)
  adding: results/7.wav (deflated 10%)
  adding: results/2.wav (deflated 7%)
  adding: results/5.wav (deflated 15%)
  adding: results/fearless_8.wav (deflated 10%)
  adding: results/13.wav (deflated 21%)
  adding: results/10.wav (deflated 13%)
  adding: results/fearless_9.wav (deflated 11%)
  adding: results/6.wav (deflated 13%)
  adding: results/9.wav (deflated 13%)
  adding: results/0.wav (deflated 18%)
  adding: results/fearless_1.wav (deflated 10%)
  adding: results/1.wav (deflated 9%)
  adding: results/fearless_5.wav (deflated 12%)
  adding: results/fearless_10.wav (deflated 11%)
  

In [ ]:
#@markdown # Delete old inputted wav and rendered wav

#@markdown ___

#@markdown Run this cell if you want to redo the process, this cell will flush every .wav in results folder and batch_audio folder
!rm -rf /content/diff-svc/batch_audio/*.wav
!rm -rf /content/diff-svc/results/*.wav